In [1]:
import numpy as np
import pandas as pd

In [401]:
n_moment = np.random.normal(0,1,1)
X = [1,2,3,4]
X = np.add(X,n_moment)
print(X)

[-0.32230035  0.67769965  1.67769965  2.67769965]


In [421]:
A = np.linspace(1,1,5) 
B = np.linspace(1,1,5) 
D = np.dot(A,B)
#print(len(D) == 1)
print(D, np.shape(D),type(D))

5.0 () <class 'numpy.float64'>


In [476]:
class rls_algo:
    def __init__(self,total_measurement,memory_size,forgetting_factor):
        self.total_measurement = total_measurement
        self.memory_size = memory_size
        self.time = 0
        self.measure = np.zeros(memory_size)
        assert(forgetting_factor < 1) # forgetting factor should be 0<x<1
        self.forgetting_factor = forgetting_factor
        self.forgetting_factor_matrix = np.array([(1 ,0, 0),(0, forgetting_factor, 0),(0, 0, forgetting_factor**2)])

        #self.channels = np.linspace(1,0.2,memory_size) #Fading channel
        self.channels = np.linspace(1,1,memory_size) 
        self.h =  np.ones(memory_size)                    # Estimation vector

        # Initiate neccesery matrices
        self.q_test     = np.zeros(memory_size)            #Dimension is (,mem)
        self.p_test     = np.zeros(memory_size)            #Dimension is (,mem)
        self_Rx_martrix = np.zeros((memory_size,memory_size)) #Dimension is (mem,mem)
        self_P          = np.zeros((memory_size,memory_size)) #Dimension is (mem,mem)
        self.k          = np.zeros(memory_size)            #Dimension is (,mem)

        self.x = 0
        self.y = 0

    # Update the measurments
    #####################################################
    def update_measurments(self):
        self.time = self.time + 1
        self.measure = np.array(self.total_measurement[self.time:self.time+3])

    #Generate x with new measurements
    #####################################################
    def generate_x(self):
        self.x  = np.convolve(self.channels,self.measure,"valid")
        #self.x  = np.dot(self.channels,self.measure)
        assert(len(self.x) == 1) #To make any prediction the "amount of measurements" aquired must be at "number of channels".
        n_moment = np.random.normal(0,1,1)
        self.x = np.add(self.x,n_moment)

        #q_test array is filled as FILO
        self.q_test = np.insert( self.q_test ,0,self.x)
        self.q_test = np.delete( self.q_test ,(len(self.q_test)-1))

    #Generate y with x and previous estimates
    #####################################################
    def generate_y(self):
        #Determine y and e append to Y
        self.y = np.convolve(self.q_test,self.h,"valid")
        #self.y  = np.dot(self.h,self.q_test)
        assert(len(self.y) == 1) #To make any prediction the "amount of measurements" aquired must be at "number of channels".
    
    #Determine p array
    #####################################################
    def update_p(self):
        assert(np.shape(self.p_test) == (3,)) # p_test is not correct shape
        if (self.time > self.memory_size):
            self.p_test = self.forgetting_factor*self.p_test + self.q_test*self.measure[2]
        elif(self.time == self.memory_size):
            # First meanigful p is determined after num of recorded measurements are equal to num of memory
            for k in range (self.memory_size):
                self.p_test =  np.zeros(self.memory_size)
                self.p_test =  self.p_test + np.transpose(self.q_test)*self.measure[k]*self.forgetting_factor**self.k
        else: 
            pass

    #Function for determining R_x matrix recursively
    #####################################################
    def update_R_x(self):
        if (self.time > self.memory_size):
            self.R_x_matrix = self.forgetting_factor*self.R_x_matrix + np.outer(np.transpose(self.q_test), self.q_test)
        elif(self.time == self.memory_size):
            # First meanigful R_x is determined after num of recorded measurements are equal to num of memory
            self.R_x_matrix =  np.outer(np.transpose(self.q_test), self.q_test) * self.forgetting_factor_matrix
        else: 
            pass

    #Function for updating { P  == inverse of R_x } recursively
    #####################################################
    def update_P(self):
        if (self.time > self.memory_size):    
            #Determing  Kalman Gain Factor
            self.k = np.dot(self.P,self.q_test)/(self.forgetting_factor+ np.dot(np.dot(np.transpose(self.q_test),self.P),np.transpose(self.q_test)))
            # Sherman-Morrison formula for updating the inverse of R_x
            self.P = (self.forgetting_factor**-1)*self.P -((self.forgetting_factor**-1)* np.dot(self.k,np.transpose(self.q_test))*self.P)
        elif(self.time == self.memory_size):
            # First meanigful P is determined after num of recorded measurements are equal to num of memory
            assert(np.linalg.det(self.R_x_matrix) != 0) 
            # Zero determinant matrix inversion not possible wait for another measurement
            self.P = np.linalg.inv(self.R_x_matrix)
        else: 
            pass

    #Function for updating {-- The Estimate --} recursively
    #####################################################
    def update_solution(self):
        if (self.time > self.memory_size):
            self.h = self.h + self.k*(self.measure -np.transpose(self.q_test)*self.h) 
        elif(self.time == self.memory_size):
            pass
        else: 
            pass


    #  Reset variables (Does not work correctly ?)
    #####################################################
    def reset_timer(self):
        self.time    = 0
        self.measure = np.zeros(self.memory_size)
        self.x = 0
        self.y = 0
        self.q_test     = np.zeros(self.memory_size)            
        self.p_Test     = np.zeros(self.memory_size)     
        self_Rx_martrix = np.zeros((self.memory_size,self.memory_size)) #Dimension is (mem,mem)
        self.k          = np.zeros(self.memory_size)            #Dimension is (,mem)       
        self_P          = np.zeros((self.memory_size,self.memory_size)) #Dimension is (mem,mem)
        self.h          =  np.ones(self.memory_size)                    # Estimation vector

        

In [438]:
full_measurments = np.append(np.array([4,2,7]) ,np.arange(20) )

In [486]:
My_algo = rls_algo(total_measurement =full_measurments, memory_size=3,forgetting_factor=0.3 )

In [487]:
X_store = np.zeros((len(My_algo.total_measurement)  , 10))
Y_store = np.zeros((len(My_algo.total_measurement)  , 10))
E_store = np.zeros((len(My_algo.total_measurement)  , 10))  
H_store = np.zeros((My_algo.memory_size,10))

for k in range (0,10):
    for i in range (0, len(My_algo.total_measurement) - My_algo.memory_size ):
        My_algo.update_measurments()
        My_algo.generate_x()
        X_store[i,k] = My_algo.x
        My_algo.generate_y()
        #print("Y", My_algo.y , "X", My_algo.x ,"Measurment" ,My_algo.measure ,"H" ,My_algo.h )
        Y_store[i,k] = My_algo.y
        My_algo.update_p()
        My_algo.update_R_x()
        My_algo.update_P()
        My_algo.update_solution()
        #print(My_algo.time)

    print("Estimation no {} -->".format(k), My_algo.h)
    H_store[:,k] = My_algo.h
    E_store[:,k] = Y_store[:,k] - My_algo.total_measurement
    My_algo = rls_algo(total_measurement =full_measurments, memory_size=3,forgetting_factor=0.3 )

Estimation no 0 --> [0.30761366 0.4984518  0.42100682]
Estimation no 1 --> [0.31132497 0.5055751  0.40641144]
Estimation no 2 --> [0.30961732 0.69153345 0.51486928]
Estimation no 3 --> [0.30962811 0.40449379 0.40643099]
Estimation no 4 --> [0.31573984 0.45791352 0.44553463]
Estimation no 5 --> [0.31332599 0.4143912  0.40685894]
Estimation no 6 --> [0.31369367 0.37610697 0.40964208]
Estimation no 7 --> [0.30926182 0.47795608 0.40796828]
Estimation no 8 --> [0.31504696 0.42372632 0.40376172]
Estimation no 9 --> [0.31269534 0.64487839 0.47220255]


In [488]:
E_store[:,2] #Error is increasing constantly why ??

array([  6.68846325,  18.15124059,  14.67357573,  16.03257121,
        11.306401  ,  16.96609926,  22.80947845,  27.89768195,
        30.37664506,  33.71607075,  38.06139516,  40.94718547,
        44.82619112,  48.38833099,  50.91847124,  53.34690108,
        55.16254387,  57.40632368,  61.00396957,  64.69011718,
       -17.        , -18.        , -19.        ])

In [489]:
print(Y_store[10:15,2])
print(X_store[10:15,2])

[45.06139516 48.94718547 53.82619112 58.38833099 61.91847124]
[26.83644187 28.48873218 34.09991038 35.41214768 38.44596361]


In [ ]:
#### Debugging 
    # X,Y generation

In [471]:
class generate_sm():
    def __init__(self,channels,measurments,mem_size):
        self.channels    = channels
        self.measurments = measurments
        self.x = 0
        self.y = 0
        self.q_test = np.zeros(mem_size)

    #Generate x with new measurements
    #####################################################
    def generate_x(self):
        self.x  = np.convolve(self.channels,self.measurments,"valid")
        #assert(len(self.x) == 1) #To make any prediction the "amount of measurements" aquired must be at "number of channels".
        n_moment = np.random.normal(0,1,1)
        self.x = np.add(self.x,n_moment)

        #q_test array is filled as FILO
        self.q_test = np.insert( self.q_test ,0,self.x)
        self.q_test = np.delete( self.q_test ,(len(self.q_test)-1))

    #Generate y with x and previous estimates
    #####################################################
    def generate_y(self):
        #Determine y and e append to Y
        self.y = np.convolve(h,self.q_test,"valid")
        #assert(len(self.y) == 1) #To make any prediction the "amount of measurements" aquired must be at "number of channels".


In [472]:
My_test = generate_sm(channels=[1,2,3],measurments=np.arange(10),mem_size = 3)

In [473]:
My_test.generate_x()

In [474]:
My_test.x

array([ 4.59632062, 10.59632062, 16.59632062, 22.59632062, 28.59632062,
       34.59632062, 40.59632062, 46.59632062])